In [257]:
import sqlite3

# CREATE FIRST TABLE
population_data = []

with open('World_country_populations.csv') as file:
    for line in file:
        line = line.strip().split(',')
        line[1] = line[1][:-1]
        population_data.append(line)


connection = sqlite3.connect('country_data.db')
cursor = connection.cursor()
cursor.execute('create table if not exists countries (num int, country, population int, migrants int, median_age int)')

for country in population_data[1:]:
    cursor.execute('insert into countries values(?,?,?,?,?)',(int(country[0]), country[1], int(country[2]), country[7], country[9]))
    
connection.commit()
connection.close()

In [258]:
import sqlite3

# CREATE SECOND TABLE
gdp_data = []

with open('GDP_countries.txt') as file:
    
    for line in file:
        if 'Estimate' in line:
            continue
        line = line.strip().split('\t')
        while len(line) >= 4:
            # if the year has extra characters
            if len(line[-1]) > 4:
                # remove them
                line[-1] = line[-1][5:]

            # Replacing n/a entries with a zero
            if 'n/a' in line[-2].lower():
                line[-2] = 0
            # if
            try:
                line[-2] = int(line[-2].replace(',',''))
                line[-1] = int(line[-1])
            except:
                pass     
            gdp_data.append(line[:2]+line[-1:-3:-1])
            line.pop()
            line.pop()
connection = sqlite3.connect('country_data.db')
cursor = connection.cursor()
cursor.execute('create table if not exists gdp_countries (country_Territory, region, year int, estimate int)')

for country in gdp_data:
    cursor.execute('insert into gdp_countries values(?,?,?,?)',country)

connection.commit()
connection.close()

In [337]:
# Calculate the population range
# Find countries at the bottom 20%
# Find total population of the bottom 50%
# Find the mean population

connection = sqlite3.connect('country_data.db')
cursor = connection.cursor()
data = list(cursor.execute('select * from countries order by population desc'))
connection.close()

# Countries within lowest 20% population
countries_bottom_20 = (int(len(data)*.2))
print('Countries within lowest 20% population:')
print(*data[:-countries_bottom_20:-1],'\n')

# Combined population of the bottom 50%
countries_bottom_50 = (int(len(data)*.5))
total = 0

for country in range(-1,-countries_bottom_50,-1):
    total += data[country][2]
print(f"Combined population of the bottom 50% {total}\n")

# Find mean population of all countries
total = 0
for country in data:
    total += country[2]
print(f'mean population of all countries {total/len(data)}\n')

Countries within lowest 20% population:
(235, 'Holy See', 801, '', 'N.A. ') (234, 'Tokelau', 1357, '', 'N.A. ') (233, 'Niue', 1626, '', 'N.A. ') (232, 'Falkland Islands', 3480, '', 'N.A. ') (231, 'Montserrat', 4992, '', 'N.A. ') (230, 'Saint Pierre & Miquelon', 5794, '', 'N.A. ') (229, 'Saint Helena', 6077, '', 'N.A. ') (228, 'Saint Barthelemy', 9877, '', 'N.A. ') (227, 'Nauru', 10824, '', 'N.A. ') (226, 'Wallis & Futuna', 11239, '', 'N.A. ') (225, 'Tuvalu', 11792, '', 'N.A. ') (224, 'Anguilla', 15003, '', 'N.A. ') (223, 'Cook Islands', 17564, '', 'N.A. ') (222, 'Palau', 18094, '', 'N.A. ') (221, 'Caribbean Netherlands', 26223, '', 'N.A. ') (220, 'British Virgin Islands', 30231, '', 'N.A. ') (219, 'Gibraltar', 33691, '', 'N.A. ') (218, 'San Marino', 33931, '', 'N.A. ') (217, 'Liechtenstein', 38128, '', 'N.A. ') (216, 'Saint Martin', 38666, '', 'N.A. ') (215, 'Turks and Caicos', 38717, '', 'N.A. ') (214, 'Monaco', 39242, '', 'N.A. ') (213, 'Sint Maarten', 42876, '', 'N.A. ') (212, 'Faer

In [326]:
import statistics

connection = sqlite3.connect('country_data.db')
cursor = connection.cursor()

data_all = list(cursor.execute('select * from gdp_countries order by estimate desc'))
population_data = list(cursor.execute('select * from countries order by population desc'))
data_2021 = list(cursor.execute('select distinct * from gdp_countries where year == 2021 order by estimate desc'))
connection.close()

# 5 - Using the statistics library what is the standard deviation of the population data
population_all = []
for country in population_data:
    population_all.append(country[2])
    total += country[2]
standard_deviation_all = round(statistics.stdev(population_all),2)
print(f'Standard Deviation All Population: {standard_deviation_all}\n')


# 6 - How many countries are within 1 standard deviation of the mean (incomplete)
total = 0
gdp_2021 = []
for country in data_2021:
    gdp_2021.append(country[-1])
    total += country[-1]

mean_2021 = statistics.mean(population_all)
print(round(mean_2021, 2))


# 7 - Display the country name, population, and GDP for 2021
connection = sqlite3.connect('country_data.db')
cursor = connection.cursor()
cursor.execute(f'SELECT A.country, A.population, B.estimate, B.year FROM countries A LEFT JOIN gdp_countries B on A.country = B.country_Territory order by A.population desc')
rows = cursor.fetchall()

print('Country Name, Population , and GDP for 2021:')
for country in data_2021:
    print('\t',country)


# 8 - Print the top 10 countries GDP per population (incomplete)
gdp_per_person = []
for row in rows:
    try:
        gdp_per_person.append([row[2]/row[1], row[0]])
    except:
        pass
[print(person) for person in sorted(gdp_per_person, reverse = True)[:10]]

# 9 - Print the bottom 50 countries GDP per population
items_found = 0
counter = 0
while items_found < 50:
    if isinstance(rows[counter][1], int) and isinstance(rows[counter][2], int) and isinstance(rows[counter][3], int):
        items_found += 1
        print(rows[counter])
    counter -= 1

Standard Deviation All Population: 135136985.28

33171202.68
Country Name, Population , and GDP for 2021:
	 ('United States', 'Americas', 2021, 22675271)
	 ('China', 'Asia', 2021, 16642318)
	 ('Japan', 'Asia', 2021, 5378136)
	 ('Germany', 'Europe', 2021, 4319286)
	 ('United Kingdom', 'Europe', 2021, 3124650)
	 ('India', 'Asia', 2021, 3049704)
	 ('France', 'Europe', 2021, 2938271)
	 ('Italy', 'Europe', 2021, 2106287)
	 ('Canada', 'Americas', 2021, 1883487)
	 ('South Korea', 'Asia', 2021, 1806707)
	 ('Russia', 'Europe', 2021, 1710734)
	 ('Australia', 'Oceania', 2021, 1617543)
	 ('Brazil', 'Americas', 2021, 1491772)
	 ('Spain', 'Europe', 2021, 1461552)
	 ('Mexico', 'Americas', 2021, 1192480)
	 ('Indonesia', 'Asia', 2021, 1158783)
	 ('Netherlands', 'Europe', 2021, 1012598)
	 ('Switzerland', 'Europe', 2021, 824734)
	 ('Saudi Arabia', 'Asia', 2021, 804921)
	 ('Turkey', 'Asia', 2021, 794530)
	 ('Taiwan', 'Asia', 2021, 759104)
	 ('Iran', 'Asia', 2021, 682859)
	 ('Poland', 'Europe', 2021, 64212